# Tests for tinwidgets library
To run a test just exectute a cell and test things. 
Keep in mind that there is only a figure in the entire notebook (this is done to reduce lag), so only one test can be executed at a time.

## Using blitting with widget backend
If something is wrong make sure:
- to be using ```with plt.ioff():
    fig, ax = plt.subplots()``` when opening a figure.
    Two figures may be displayed instead
- to be calling ```ax.add_artist``` only one time. After that you can use ```ax.draw_artist``` to render it

In [6]:
'''
STANDALONE EXAMPLE OF A DRAGGABLE AND RESIZABLE LINE
'''
import matplotlib.pyplot as plt
from matplotlib import transforms
from matplotlib.lines import Line2D
import ipywidgets as widgets
import numpy as np
import test_utils
# This is important!
test_utils.prepare_env()

# ___ EVENT HANDLERS ___
def update_bg():
    global bg
    fig.canvas.draw()
    bg = fig.canvas.copy_from_bbox(ax.bbox)

def on_pick(event):
    art = event.artist
    if art == line:
        line.set_animated(True)
        line.set_color('red')
        update_bg()
        ax.draw_artist(line)
        fig.canvas.blit(ax.bbox)
        # the animated artists are excluded in the normal 
        # figure rendering
    
def on_move(event):
    # ___ FRAME SKIPPING LOGIC ____
    if on_move.skip_frames < 0:
        on_move.skip_frames = 0
    try:
        if on_move.frame_skip == 'normal':
            if on_move.skip_frames:
                on_move.skip_frames -= 1
                return
            on_move.skip_frames = SKIP_FRAMES
            
        elif on_move.frame_skip == 'inverted':
            if not on_move.skip_frames:
                on_move.skip_frames = SKIP_FRAMES
                return
            on_move.skip_frames -= 1
        else:
            print('The mode is invalid')
            raise RuntimeError
    except:
        raise RuntimeError
    
    # ___ LOGIC ___
    # fetch relevant data and update state
    x, y = event.xdata, event.ydata
    px, py = on_move.last
    on_move.last = (x, y)
    modifiers = event.modifiers

    # if line is not animated why bother
    if not line.get_animated():
        return

    # exit any of the variables is none,
    # this can occur if mouse is out of 
    # the axis
    if not (x and y and px and py):
        return
    
    dx = x - px
    dy = y - py
    
    # modifiers can change the bahaviour from
    # a translation to a scale
    if 'ctrl' in modifiers:
        line.set_transform(transforms.Affine2D().scale(1 + dx, 1 + dy) + line.get_transform())
    else:
        # WARNING: Transform.__add__ is NOT commutative!
        # this means that .translate + .get_transform is not the 
        # same as .get_transform + .translate
        # Compute the current scale factors from the transformation
        # Extract the current transformation matrix
        line.set_transform(transforms.Affine2D().translate(dx, dy) + line.get_transform())
    
    global bg
    if bg is None:
        bg = fig.canvas.copy_from_bbox(ax.bbox)   
    fig.canvas.restore_region(bg)
    ax.draw_artist(line)
    fig.canvas.blit(ax.bbox)

def on_release(event):
    if line.get_animated():
        line.set_animated(False)
        line.set_color('blue')  # Reset line color
        ax.draw_artist(line)
        update_bg()
        # fig.canvas.draw_idle()  # Trigger a full redraw to finalize changes

def on_draw(event):
    # When there is a redraw clear the previuos bg
    global bg
    bg = None
    
with plt.ioff():
    fig, ax = plt.subplots()  
    
x = np.linspace(0,5,30)
y = np.sin(x)
line = Line2D(x, y)
line.set_picker(5)

# adds the line to the axs objects
# when calling ax.draw() the line 
# is drawn
# NOTE: if line.animated is true then 
# ax.draw() will not draw the line,
# the updates are handled manually
ax.add_line(line)
ax.relim()
ax.autoscale_view()
fig.tight_layout()
fig.set_size_inches(5,3.5)
fig.canvas.draw()

bg = fig.canvas.copy_from_bbox(ax.bbox)
fig.canvas.blit(ax.bbox)

# initialize the static variables
SKIP_FRAMES = 0
on_move.last = (None, None)
on_move.skip_frames = SKIP_FRAMES
on_move.frame_skip = 'normal'

fig.canvas.mpl_connect('pick_event', on_pick)
fig.canvas.mpl_connect('motion_notify_event', on_move)
fig.canvas.mpl_connect('button_release_event', on_release)
fig.canvas.mpl_connect('figure_leave_event', on_release)
fig.canvas.mpl_connect('draw_event', on_draw)

# ___ WIDGETS ___
def on_value_change(change):
    global SKIP_FRAMES
    SKIP_FRAMES = change['new']

def on_check(change):
    global out
    with out:
        print(change['new'])
        on_move.frame_skip = change['new']
        
int_range = widgets.IntSlider(value=0, max=10, min=0, description = 'FrameSkip:')
check = widgets.ToggleButtons(options=['normal', 'inverted'],
                              description = 'Frame skipping mode:',
                              tooltips = ['frames skipped', 'frames drawn after every skip'])
        
int_range.observe(on_value_change, names='value')
check.observe(on_check, names='value')
out = widgets.Output()
out.append_stdout('stdout:\n')
out.append_stdout(fig.get_dpi())
right_box = widgets.VBox([check, int_range, out])
app = widgets.HBox([fig.canvas, right_box])
app


In [8]:
'''
STANDALONE EXAMPLE OF A DRAGGABLE AND RESIZABLE RECTANGLE
'''
import matplotlib.pyplot as plt
from matplotlib import transforms
from matplotlib.patches import Rectangle
import ipywidgets as widgets
import numpy as np
import test_utils 
# This is important!
test_utils.prepare_env()

# ___ EVENT HANDLERS ___
def update_bg():
    global bg
    fig.canvas.draw()
    bg = fig.canvas.copy_from_bbox(ax.bbox)

def on_pick(event):
    art = event.artist
    if art == rect:
        rect.set_animated(True)
        # rect.set_color('red')
        rect.set_linewidth(1)
        rect.set_alpha(.6)
        update_bg()
        ax.draw_artist(rect)
        fig.canvas.blit(ax.bbox)
        # the animated artists are excluded in the normal 
        # figure rendering
    
def on_move(event):
    # ___ FRAME SKIPPING LOGIC ____
    if on_move.skip_frames < 0:
        on_move.skip_frames = 0
    try:
        if on_move.frame_skip == 'normal':
            if on_move.skip_frames:
                on_move.skip_frames -= 1
                return
            on_move.skip_frames = SKIP_FRAMES
            
        elif on_move.frame_skip == 'inverted':
            if not on_move.skip_frames:
                on_move.skip_frames = SKIP_FRAMES
                return
            on_move.skip_frames -= 1
        else:
            print('The mode is invalid')
            raise RuntimeError
    except:
        raise RuntimeError
    
    # ___ LOGIC ___
    # fetch relevant data and update state
    x, y = event.xdata, event.ydata
    px, py = on_move.last
    on_move.last = (x, y)
    modifiers = event.modifiers
    
    # if line is not animated why bother
    artists = [art for art in ax.artists if art.get_animated()] 
    if not rect.get_animated():
        return

    # exit any of the variables is none,
    # this can occur if mouse is out of 
    # the axis
    if not (x and y and px and py):
        return
    
    dx = x - px
    dy = y - py
    
    # modifiers can change the bahaviour from
    # a translation to a scale
    if 'ctrl' in modifiers:
        width, height = rect.get_width(), rect.get_height()
        width += dx
        height += dy
        rect.set_width(width)
        rect.set_height(height)
    else:
        # WARNING: Transform.__add__ is NOT commutative!
        # this means that .translate + .get_transform is not the 
        # same as .get_transform + .translate
        # Compute the current scale factors from the transformation
        # Extract the current transformation matrix
        xy = np.array(rect.get_xy())
        xy = xy + [dx, dy]
        rect.set_xy(xy)
       
    
    global bg
    if bg is None:
        bg = fig.canvas.copy_from_bbox(ax.bbox)   
    fig.canvas.restore_region(bg)
    ax.draw_artist(rect)
    fig.canvas.blit(ax.bbox)

def on_release(event):
    if rect.get_animated():
        rect.set_animated(False)
        # rect.set_color('red')  # Reset line color
        rect.set_linewidth(0)
        rect.set_alpha(.3)
        ax.draw_artist(rect)
        update_bg()
        # fig.canvas.draw_idle()  # Trigger a full redraw to finalize changes

def on_draw(event):
    # When there is a redraw clear the previuos bg
    global bg
    bg = None
    
with plt.ioff():
    fig, ax = plt.subplots()  
    
x = np.linspace(0,5,30)
y = np.sin(x)
rect = Rectangle([0.5, 0.5], 5, 10, alpha=.3,
                 color = 'yellow', lw = 0)
rect.set_picker(5)

# adds the line to the axs objects
# when calling ax.draw() the line 
# is drawn
# NOTE: if line.animated is true then 
# ax.draw() will not draw the line,
# the updates are handled manually
ax.add_artist(rect)
ax.relim()
ax.autoscale_view()
fig.tight_layout()
fig.set_size_inches(5,3.5)
fig.canvas.draw()

bg = fig.canvas.copy_from_bbox(ax.bbox)
fig.canvas.blit(ax.bbox)

# initialize the static variables
SKIP_FRAMES = 0
on_move.last = (None, None)
on_move.skip_frames = SKIP_FRAMES
on_move.frame_skip = 'normal'

fig.canvas.mpl_connect('pick_event', on_pick)
fig.canvas.mpl_connect('motion_notify_event', on_move)
fig.canvas.mpl_connect('button_release_event', on_release)
fig.canvas.mpl_connect('figure_leave_event', on_release)
fig.canvas.mpl_connect('draw_event', on_draw)

# ___ WIDGETS ___
def on_value_change(change):
    global SKIP_FRAMES
    SKIP_FRAMES = change['new']

def on_check(change):
    global out
    with out:
        print(change['new'])
        on_move.frame_skip = change['new']
        
int_range = widgets.IntSlider(value=0, max=10, min=0, description = 'FrameSkip:')
check = widgets.ToggleButtons(options=['normal', 'inverted'],
                              description = 'Frame skipping mode:',
                              tooltips = ['frames skipped', 'frames drawn after every skip'])
        
int_range.observe(on_value_change, names='value')
check.observe(on_check, names='value')
out = widgets.Output()
out.append_stdout('stdout:\n')
out.append_stdout(fig.get_dpi())
right_box = widgets.VBox([check, int_range, out])
app = widgets.HBox([fig.canvas, right_box])

app

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
'''
MULTIPLE RECTANGLES
This example focuses on user interactions where
there are multiple aritst present in the screen 
at once.
The general tought is that only one artist should 
be able to be picked at a time.

In particular changes are made to the on_pick function 
and on_move function.
'''
import matplotlib.pyplot as plt
from matplotlib import transforms
from matplotlib.patches import Rectangle
import ipywidgets as widgets
import numpy as np
import tests.test_utils as test_utils

# THIS IS IMPORTANT!
test_utils.prepare_env()

# ___ EVENT HANDLERS ___
def update_bg():
    global bg
    fig.canvas.draw()
    bg = fig.canvas.copy_from_bbox(ax.bbox)

def on_pick(event):
    art = event.artist
    
    # If there is an artist picked 
    # just drop out
    if not on_pick.current is None:
        return
    else:
        on_pick.current = art
        
    with out:
        print(f'picked artist {type(art)} id:{id(art)}')
        
    art.set_animated(True)
    # rect.set_color('red')
    art.set_linewidth(1)
    art.set_alpha(.6)
    update_bg()
    ax.draw_artist(art)
    fig.canvas.blit(ax.bbox)
    # the animated artists are excluded in the normal 
    # figure rendering
    
def on_move(event): 
    # ___ FRAME SKIPPING LOGIC ____
    if on_move.skip_frames < 0:
        on_move.skip_frames = 0
    try:
        if on_move.frame_skip == 'normal':
            if on_move.skip_frames:
                on_move.skip_frames -= 1
                return
            on_move.skip_frames = SKIP_FRAMES
            
        elif on_move.frame_skip == 'inverted':
            if not on_move.skip_frames:
                on_move.skip_frames = SKIP_FRAMES
                return
            on_move.skip_frames -= 1
        else:
            print('The mode is invalid')
            raise RuntimeError
    except:
        raise RuntimeError
    
    # ___ LOGIC ___
    
    # Fetch relevant data and update state,
    # THIS MUST BE DONE EVEN IF NO ARTIST IS SELECTED
    # because when you click the canvas the previous 
    # position of the mouse must be known.
    # A possible optimization is to update on_move.last 
    # the first time when the object is picked, but this 
    # adds unwanted complexity to the logic.
    # Functions should have minimum scope and responsability.
    # Its not good that on_pick should care about movement...
    # on_move should!
    x, y = event.xdata, event.ydata
    px, py = on_move.last
    on_move.last = (x, y)
    modifiers = event.modifiers
    
    # If no artist is picked why bother updating
    if on_pick.current is None:
        return
    # The class is explicitly stated
    # to get autocomplete.
    art : Rectangle = on_pick.current
    
    # Paranoia: if the artist is not animated
    # something when orribly wrong
    if not art.get_animated():
        raise RuntimeError("The artist is not animated")
    
    # exit any of the variables is None,
    # this can occur if mouse is out of 
    # the axis
    if not (x and y and px and py):
        return

    dx = x - px
    dy = y - py
    
    # modifiers can change the bahaviour from
    # a translation to a scale
    if 'ctrl' in modifiers:
        width, height = art.get_width(), art.get_height()
        width += dx
        height += dy
        art.set_width(width)
        art.set_height(height)
    else:
        # WARNING: Transform.__add__ is NOT commutative!
        # this means that .translate + .get_transform is not the 
        # same as .get_transform + .translate
        # Compute the current scale factors from the transformation
        # Extract the current transformation matrix
        xy = np.array(art.get_xy())
        xy = xy + [dx, dy]
        art.set_xy(xy)
       
    
    global bg
    if bg is None:
        bg = fig.canvas.copy_from_bbox(ax.bbox)   
    fig.canvas.restore_region(bg)
    ax.draw_artist(art)
    fig.canvas.blit(ax.bbox)

def on_release(event):
    if on_pick.current is None:
        return
    
    # Reset picked artist state
    art = on_pick.current
    art.set_animated(False)
    art.set_linewidth(0)
    art.set_alpha(.3)
    ax.draw_artist(art)
    update_bg()
    # fig.canvas.draw_idle()  # Trigger a full redraw to finalize changes
    
    # RESETTING SHOULD BE LAST
    # This is because art = on_pick.current
    # is done for convenience but now its a 
    # reference to on_pick.current.
    # Setting current to None also sets art to None.
    with out:
        print(f'released artist {type(art)} id:{id(art)}')
    on_pick.current = None

def on_draw(event):
    # When there is a redraw clear the previuos bg
    global bg
    bg = None
    
with plt.ioff():
    fig, ax = plt.subplots()  
    
x = np.linspace(0,5,30)
y = np.sin(x)
rect = Rectangle([0.5, 0.5], 5, 10, alpha=.3,
                 color = 'yellow', lw = 0)
rect1 = Rectangle([1, 1], 5, 10, alpha=.3,
                 color = 'yellow', lw = 0)
rect.set_picker(5)
rect1.set_picker(5)

# adds the line to the axs objects
# when calling ax.draw() the line 
# is drawn
# NOTE: if line.animated is true then 
# ax.draw() will not draw the line,
# the updates are handled manually
ax.add_artist(rect)
ax.add_artist(rect1)
ax.relim()
ax.autoscale_view()
fig.tight_layout()
fig.set_size_inches(5,3.5)
fig.canvas.draw()

bg = fig.canvas.copy_from_bbox(ax.bbox)
fig.canvas.blit(ax.bbox)

# initialize the static variables
SKIP_FRAMES = 0
on_move.last = (None, None)
on_move.skip_frames = SKIP_FRAMES
on_move.frame_skip = 'normal'
on_pick.current = None

fig.canvas.mpl_connect('pick_event', on_pick)
fig.canvas.mpl_connect('motion_notify_event', on_move)
fig.canvas.mpl_connect('button_release_event', on_release)
fig.canvas.mpl_connect('figure_leave_event', on_release)
fig.canvas.mpl_connect('draw_event', on_draw)

# ___ WIDGETS ___
def on_value_change(change):
    global SKIP_FRAMES
    SKIP_FRAMES = change['new']

def on_check(change):
    global out
    with out:
        print(change['new'])
        on_move.frame_skip = change['new']
        
int_range = widgets.IntSlider(value=0, max=10, min=0, description = 'FrameSkip:')
check = widgets.ToggleButtons(options=['normal', 'inverted'],
                              description = 'Frame skipping mode:',
                              tooltips = ['frames skipped', 'frames drawn after every skip'])
        
int_range.observe(on_value_change, names='value')
check.observe(on_check, names='value')
out = widgets.Output()
out.append_stdout('stdout:\n')
out.append_stdout(fig.get_dpi())
right_box = widgets.VBox([check, int_range, out])
app = widgets.HBox([fig.canvas, right_box])

app

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
''' 
This is an experiment to find out 
how are different objects
saved in memory and differenciated.
'''

r1 = Rectangle([0.5, 0.5], 1, 1)
r2 = Rectangle([0.5, 0.5], 1, 1)
r3 = r1
r4 = Rectangle([0.6, 0.6], 1, 1)
print(f'r1 is r1 {r1 is r1}')
print(f'r1 is r2 {r1 is r2}')
print(f'r1 is r3 {r1 is r3}')
print(f'r1 is r4 {r1 is r4}')
print(f'r1 id: {id(r1)}')
print(f'r2 id: {id(r2)}')
print(f'r3 id: {id(r3)}')
print(f'r4 id: {id(r4)}')


r1 is r1 True
r1 is r2 False
r1 is r3 True
r1 is r4 False
r1 id: 2132615021344
r2 id: 2132550924720
r3 id: 2132615021344
r4 id: 2132592563776


In [41]:
''' 
Test to figure out type checking
'''
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
from matplotlib.artist import Artist

r1 = Rectangle([0.5, 0.5], 1, 1)
l1 = Line2D([0, 1, 2, 3], [0, 1, 2, 3])

print(f'is r1 a Rectangle? {r1 is Rectangle}')
print(f'is r1 an Artist? {r1 is Artist}')
print(f'is r1 of type Rectangle? {type(r1) is Rectangle}')
print(f'is r1 of type Artist? {type(r1) is Artist}')
print(f'is r1 instance of Rectangle? {isinstance(r1, Rectangle)}')
print(f'is r1 instance of Artist? {isinstance(r1, Artist)}')
print(f'is r1 an instance of Line2D? {isinstance(r1, Line2D)}')

print(f'is l1 an instance of Rectangle? {isinstance(l1, Rectangle)}')
print(f'is l1 an instance of Line2D? {isinstance(l1, Line2D)}')


is r1 a Rectangle? False
is r1 an Artist? False
is r1 of type Rectangle? True
is r1 of type Artist? False
is r1 instance of Rectangle? True
is r1 instance of Artist? True
is r1 an instance of Line2D? False
is l1 an instance of Rectangle? False
is l1 an instance of Line2D? True


In [54]:
''' 
Test to see if a dict with classes works
'''

from matplotlib.patches import Rectangle, Circle
from matplotlib.lines import Line2D
from matplotlib.artist import Artist

r1 = Rectangle([0.5, 0.5], 1, 1)
l1 = Line2D([0, 1, 2, 3], [0, 1, 2, 3])
random = None

styles = {
    Artist : 'artist syle',
    Rectangle : 'rect style',
    Line2D : 'line style'
}

print(f'style for r1 is {styles.get(type(r1))}')
print(f'style for l1 is {styles.get(type(l1))}')
print(f'style for None is {styles.get(None)}')
# Usin styles[None] is preferred since it throws a 
# keyerror that helps catch bugs
try:
    print(f'style for None is {styles[None]}')
except KeyError as e:
    print(f'The style for {e} is not supported')

try:
    print(f'style for None is {styles[Circle]}')
except KeyError as e:
    print(f'The style for {e} is not supported')

style for r1 is rect style
style for l1 is line style
style for None is None
The style for None is not supported
The style for <class 'matplotlib.patches.Circle'> is not supported


In [9]:
''' 
Try to set attributes from dict
'''
from matplotlib.patches import Rectangle
style = {
    'alpha' : 0.5,
    'lw' : 1
}

rect = Rectangle([0.5, 0.5], 1, 1)
for key, value in style.items():
    print(key, value)
    rect.set(key = value)


alpha 0.5


AttributeError: Rectangle.set() got an unexpected keyword argument 'key'

In [2]:
'''
MULTIPLE ARTISTS
This example focuses on user interactions where
there are multiple rectangles present in the screen 
at once.
The general tought is that only one artist should 
be able to be picked at a time.

In particular changes are made to the on_pick function
and on_move function.

Now the on_pick function also saves the properties of 
the artist and on_release resets them.
The on_move functions now updates the artist depending
on the type of artist selected.
'''
import matplotlib.pyplot as plt
from matplotlib import transforms
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
from matplotlib.artist import Artist
import ipywidgets as widgets
import numpy as np
import test_utils
# This is important!
test_utils.prepare_env()

# ___ EVENT HANDLERS ___
def update_bg():
    global bg
    fig.canvas.draw()
    bg = fig.canvas.copy_from_bbox(ax.bbox)

def on_pick(event):
    art : Artist = event.artist
    print(art.get_transform().get_matrix())
    # If there is an artist picked 
    # just drop out
    if not on_pick.current is None:
        return
    else:
        on_pick.current = art
        
    with out:
        print(f'picked artist {type(art)} id:{id(art)}')
        
    art.set_animated(True)
    try:
        # This saves the state of the artist properties in the 
        # on_release.style dict.
        # First all the properties are saved because there is not
        # a art.get(prop) method like there is for art.set(prop = value).
        # Then only the ones changed during pick are actually saved in
        # on_release.style
        props = art.properties()
        on_release.style = {
            key : props[key] for key in on_pick.styles[type(art)].keys()
        }
        # on_pick.styles[type(art)] is a dict with
        # all the styles needed, the ** operator
        # unpacks the dict.
        # Accessing a dict using [] throws a KeyError
        # if the key is not found.
        art.set(**on_pick.styles[type(art)])
    except KeyError as e:
        with out:
            print(f'Style for {e} is not supported')
   
    update_bg()
    ax.draw_artist(art)
    fig.canvas.blit(ax.bbox)
    # the animated artists are excluded in the normal 
    # figure rendering

def update_Rectangle(rect : Rectangle, *args):
    ''' 
    Updates the position of a dragged Rectangle artist.
    THIS IS A HARD CHANGE: the rectangle is intended to be used as a
    "selector", so its actually moved during an update and not only
    "shown" displaced at drawtime.
    '''
    dx, dy, modifiers = args
    
    # modifiers can change the bahaviour from
    # a translation to a scale
    if 'ctrl' in modifiers:
        width, height = rect.get_width(), rect.get_height()
        width += dx
        height += dy
        rect.set_width(width)
        rect.set_height(height)
    else:
        xy = np.array(rect.get_xy())
        xy = xy + [dx, dy]
        rect.set_xy(xy)

def update_Line2D(line : Line2D, *args):
    ''' 
    Updates the position of a dragged Line2D artist.
    THIS IS A SOFT CHANGE: THE ACTUAL POINTS ARE NOT 
    TOUCHED, THE ARTIST IS TRANSLATED ONLY AT DRAW TIME!
    This is important because one might not want to actually
    change the experimental wavedata (Igor like behaviuour) when dragging
    things around...
    '''
    dx, dy, modifiers = args
    
    if 'ctrl' in modifiers:
        line.set_transform(transforms.Affine2D().scale(1 + dx, 1 + dy) + line.get_transform())
    else:
        # WARNING: Transform.__add__ is NOT commutative!
        # this means that .translate + .get_transform is not the 
        # same as .get_transform + .translate
        # Compute the current scale factors from the transformation
        # Extract the current transformation matrix
        line.set_transform(transforms.Affine2D().translate(dx, dy) + line.get_transform())

def on_move(event): 
    # ___ FRAME SKIPPING LOGIC ____
    if on_move.skip_frames < 0:
        on_move.skip_frames = 0
    try:
        if on_move.frame_skip == 'normal':
            if on_move.skip_frames:
                on_move.skip_frames -= 1
                return
            on_move.skip_frames = SKIP_FRAMES
            
        elif on_move.frame_skip == 'inverted':
            if not on_move.skip_frames:
                on_move.skip_frames = SKIP_FRAMES
                return
            on_move.skip_frames -= 1
        else:
            print('The mode is invalid')
            raise RuntimeError
    except:
        raise RuntimeError
    
    # ___ LOGIC ___
    
    # Fetch relevant data and update state,
    # THIS MUST BE DONE EVEN IF NO ARTIST IS SELECTED
    # because when you click the canvas the previous 
    # position of the mouse must be known.
    # A possible optimization is to update on_move.last 
    # the first time when the object is picked, but this 
    # adds unwanted complexity to the logic.
    # Functions should have minimum scope and responsability.
    # Its not good that on_pick should care about movement...
    # on_move should!
    x, y = event.xdata, event.ydata
    px, py = on_move.last
    on_move.last = (x, y)
    modifiers = event.modifiers
    
    # If no artist is picked why bother updating
    if on_pick.current is None:
        return
    # The class is explicitly stated
    # to get autocomplete.
    art : Artist = on_pick.current
    
    # Paranoia: if the artist is not animated
    # something when orribly wrong
    if not art.get_animated():
        raise RuntimeError("The artist is not animated")
    
    # exit any of the variables is None,
    # this can occur if mouse is out of 
    # the axis
    if not (x and y and px and py):
        return

    dx = x - px
    dy = y - py
    
    # This is a little bit Hacky, a little bit hard to read but simple
    # to understand: depending on the type of the artists get the 
    # appropriate update_artist function and pass to it the data 
    # that it needs
    on_move.behaviours.get(type(art))(art, dx, dy, modifiers)
    
    global bg
    if bg is None:
        bg = fig.canvas.copy_from_bbox(ax.bbox)   
    fig.canvas.restore_region(bg)
    ax.draw_artist(art)
    fig.canvas.blit(ax.bbox)

def on_release(event):
    if on_pick.current is None:
        return
    
    # Reset picked artist state
    art : Artist = on_pick.current
    art.set_animated(False)
    # Get the styles that on_pick.styles changed 
    # and restore each of them using the old
    # values in on_release.style
    try:
        art.set(**on_release.style)
    except RuntimeError as e:
        with out:
            print(e)
    
    ax.draw_artist(art)
    update_bg()
    # fig.canvas.draw_idle()  # Trigger a full redraw to finalize changes
    
    # RESETTING SHOULD BE LAST
    # This is because art = on_pick.current
    # is done for convenience but now its a 
    # reference to on_pick.current.
    # Setting current to None also sets art to None
    on_pick.current = None
    on_release.style = {}

    with out:
        print(f'released artist {type(art)} id:{id(art)}')

def on_draw(event):
    # When there is a redraw clear the previuos bg
    global bg
    bg = None
    
with plt.ioff():
    fig, ax = plt.subplots()  
    
x = np.linspace(0,5,30)
y = np.sin(x)
rect = Rectangle([0.5, 0.5], 5, 10, alpha=.3,
                 color = 'yellow', lw = 0)
rect1 = Rectangle([1, 1], 5, 10, alpha=.3,
                 color = 'yellow', lw = 0)
line = Line2D([0,1], [0,1])
rect.set_picker(5)
rect1.set_picker(5)
line.set_picker(5)
print(line.get_transform().get_matrix())

# adds the line to the axs objects
# when calling ax.draw() the line 
# is drawn
# NOTE: if line.animated is true then 
# ax.draw() will not draw the line,
# the updates are handled manually
ax.add_artist(rect)
ax.add_artist(rect1)
ax.add_line(line)
ax.relim()
ax.autoscale_view()
fig.tight_layout()
fig.set_size_inches(5,3.5)
fig.canvas.draw()

bg = fig.canvas.copy_from_bbox(ax.bbox)
fig.canvas.blit(ax.bbox)

# initialize the static variables
SKIP_FRAMES = 0
on_move.last = (None, None)
on_move.skip_frames = SKIP_FRAMES
on_move.frame_skip = 'normal'
on_move.behaviours = {
    Rectangle : update_Rectangle,
    Line2D : update_Line2D
}
on_pick.current = None
on_pick.styles = {
    Rectangle : {
        'alpha' : 0.6,
        'linewidth' : 1
    },
    Line2D : {
        'linewidth' : 2
        },
}
# This is used internally to save the style
# the artist prior to picking
on_release.style = {}

fig.canvas.mpl_connect('pick_event', on_pick)
fig.canvas.mpl_connect('motion_notify_event', on_move)
fig.canvas.mpl_connect('button_release_event', on_release)
fig.canvas.mpl_connect('figure_leave_event', on_release)
fig.canvas.mpl_connect('draw_event', on_draw)

# ___ WIDGETS ___
def on_value_change(change):
    global SKIP_FRAMES
    SKIP_FRAMES = change['new']

def on_check(change):
    global out
    with out:
        print(change['new'])
        on_move.frame_skip = change['new']
        
int_range = widgets.IntSlider(value=0, max=10, min=0, description = 'FrameSkip:')
check = widgets.ToggleButtons(options=['normal', 'inverted'],
                              description = 'Frame skipping mode:',
                              tooltips = ['frames skipped', 'frames drawn after every skip'])
        
int_range.observe(on_value_change, names='value')
check.observe(on_check, names='value')
out = widgets.Output()
out.append_stdout('stdout:\n')
out.append_stdout(fig.get_dpi())
right_box = widgets.VBox([check, int_range, out])
app = widgets.HBox([fig.canvas, right_box])

app

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


[[99.31581834  0.         77.4353724 ]
 [ 0.         36.9910483  61.00341151]
 [ 0.          0.          1.        ]]
[[99.31581834  0.         77.4353724 ]
 [ 0.         36.9910483  61.00341151]
 [ 0.          0.          1.        ]]
[[ 99.31581834   0.          81.75537257]
 [  0.          36.9910483  180.52341626]
 [  0.           0.           1.        ]]
[[ 99.31581834   0.          81.75537257]
 [  0.          36.9910483  180.52341626]
 [  0.           0.           1.        ]]
[[ 99.31581834   0.          75.99537234]
 [  0.          36.9910483  164.68341563]
 [  0.           0.           1.        ]]
[[496.5790917    0.         127.09328157]
 [  0.         369.91048302  79.49893567]
 [  0.           0.           1.        ]]
[[496.5790917    0.         176.75119074]
 [  0.         369.91048302  97.99445982]
 [  0.           0.           1.        ]]
[[496.5790917    0.         127.09328157]
 [  0.         369.91048302  79.49893567]
 [  0.           0.           1.        ]]
[[

AttributeError: 'function' object has no attribute 'skip_frames'

AttributeError: 'function' object has no attribute 'skip_frames'

AttributeError: 'function' object has no attribute 'skip_frames'

AttributeError: 'function' object has no attribute 'skip_frames'

In [1]:
''' 
Logging behaviuour test.
The logging is writtend to the an output widget 
and to a file.
'''

import logging.handlers
import ipywidgets as widgets
import logging

class OutputWidgetHandler(logging.Handler):
    """ Custom logging handler sending logs to an output widget """

    def __init__(self, *args, **kwargs):
        super(OutputWidgetHandler, self).__init__(*args, **kwargs)
        layout = {
            'width': '100%', 
            'height': '160px', 
            'border': '1px solid black'
        }
        self.out = widgets.Output(layout=layout)

    def emit(self, record):
        """ Overload of logging.Handler method """
        formatted_record = self.format(record)
        new_output = {
            'name': 'stdout', 
            'output_type': 'stream', 
            'text': formatted_record+'\n'
        }
        self.out.outputs = (new_output, ) + self.out.outputs
        
    def show_logs(self):
        """ Show the logs """
        display(self.out)
    
    def clear_logs(self):
        """ Clear the current logs """
        self.out.clear_output()


logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

widget_handler = OutputWidgetHandler()
file_handler = logging.FileHandler('debug.log')
file_handler.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s  - [%(levelname)s] %(message)s')
widget_handler.setFormatter(formatter)
file_handler.setFormatter(formatter)

logger.addHandler(widget_handler)
logger.addHandler(file_handler)

In [2]:
widget_handler.show_logs()

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [3]:
widget_handler.clear_logs()
logger.info('Starting program')

try:
    logger.info('About to try something dangerous...')
    1.0/0.0
except Exception as e:
    logger.exception('An error occurred!')

In [20]:
'''
MULTIPLE ARTISTS
This example focuses on user interactions where
there are multiple rectangles present in the screen 
at once.
The general tought is that only one artist should 
be able to be picked at a time.

In particular changes are made to the on_pick function
and on_move function.

Now the on_pick function also saves the properties of 
the artist and on_release resets them.
The on_move functions now updates the artist depending
on the type of artist selected.
'''
import matplotlib.pyplot as plt
from matplotlib import transforms
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
from matplotlib.artist import Artist
import ipywidgets as widgets
import numpy as np
import test_utils
# This is important!
test_utils.prepare_env()

# ___ EVENT HANDLERS ___
def update_bg():
    print('updating bg')
    global bg
    fig.canvas.draw()
    bg = fig.canvas.copy_from_bbox(ax.bbox)

def on_pick(event):
    print('picking')
    art : Artist = event.artist
    print(art.get_transform().get_matrix())
    # If there is an artist picked 
    # just drop out
    if not on_pick.current is None:
        return
    else:
        on_pick.current = art
        
    with out:
        print(f'picked artist {type(art)} id:{id(art)}')
        
    art.set_animated(True)
    try:
        # This saves the state of the artist properties in the 
        # on_release.style dict.
        # First all the properties are saved because there is not
        # a art.get(prop) method like there is for art.set(prop = value).
        # Then only the ones changed during pick are actually saved in
        # on_release.style
        props = art.properties()
        on_release.style = {
            key : props[key] for key in on_pick.styles[type(art)].keys()
        }
        # on_pick.styles[type(art)] is a dict with
        # all the styles needed, the ** operator
        # unpacks the dict.
        # Accessing a dict using [] throws a KeyError
        # if the key is not found.
        art.set(**on_pick.styles[type(art)])
    except KeyError as e:
        with out:
            print(f'Style for {e} is not supported')
   
    update_bg()
    ax.draw_artist(art)
    fig.canvas.blit(ax.bbox)
    # the animated artists are excluded in the normal 
    # figure rendering

def update_Rectangle(rect : Rectangle, *args):
    ''' 
    Updates the position of a dragged Rectangle artist.
    THIS IS A HARD CHANGE: the rectangle is intended to be used as a
    "selector", so its actually moved during an update and not only
    "shown" displaced at drawtime.
    '''
    print('updating rect')
    dx, dy, modifiers = args
    
    # modifiers can change the bahaviour from
    # a translation to a scale
    if 'ctrl' in modifiers:
        width, height = rect.get_width(), rect.get_height()
        width += dx
        height += dy
        rect.set_width(width)
        rect.set_height(height)
    else:
        xy = np.array(rect.get_xy())
        xy = xy + [dx, dy]
        rect.set_xy(xy)

def update_Line2D(line : Line2D, *args):
    ''' 
    Updates the position of a dragged Line2D artist.
    THIS IS A SOFT CHANGE: THE ACTUAL POINTS ARE NOT 
    TOUCHED, THE ARTIST IS TRANSLATED ONLY AT DRAW TIME!
    This is important because one might not want to actually
    change the experimental wavedata (Igor like behaviuour) when dragging
    things around...
    '''
    dx, dy, modifiers = args
    print('updating line')
    
    if 'ctrl' in modifiers:
        line.set_transform(transforms.Affine2D().scale(1 + dx, 1 + dy) + line.get_transform())
    else:
        # WARNING: Transform.__add__ is NOT commutative!
        # this means that .translate + .get_transform is not the 
        # same as .get_transform + .translate
        # Compute the current scale factors from the transformation
        # Extract the current transformation matrix
        line.set_transform(transforms.Affine2D().translate(dx, dy) + line.get_transform())

def on_move(event): 
    # ___ LOGIC ___
    
    # Fetch relevant data and update state,
    # THIS MUST BE DONE EVEN IF NO ARTIST IS SELECTED
    # because when you click the canvas the previous 
    # position of the mouse must be known.
    # A possible optimization is to update on_move.last 
    # the first time when the object is picked, but this 
    # adds unwanted complexity to the logic.
    # Functions should have minimum scope and responsability.
    # Its not good that on_pick should care about movement...
    # on_move should!
    print('moving')
    x, y = event.xdata, event.ydata
    px, py = on_move.last
    on_move.last = (x, y)
    modifiers = event.modifiers
    
    # If no artist is picked why bother updating
    if on_pick.current is None:
        return
    # The class is explicitly stated
    # to get autocomplete.
    art : Artist = on_pick.current
    
    # Paranoia: if the artist is not animated
    # something when orribly wrong
    if not art.get_animated():
        raise RuntimeError("The artist is not animated")
    
    # exit any of the variables is None,
    # this can occur if mouse is out of 
    # the axis
    if not (x and y and px and py):
        return

    dx = x - px
    dy = y - py
    
    # This is a little bit Hacky, a little bit hard to read but simple
    # to understand: depending on the type of the artists get the 
    # appropriate update_artist function and pass to it the data 
    # that it needs
    on_move.behaviours.get(type(art))(art, dx, dy, modifiers)
    
    global bg
    if bg is None:
        bg = fig.canvas.copy_from_bbox(ax.bbox)   
    fig.canvas.restore_region(bg)
    ax.draw_artist(art)
    fig.canvas.blit(ax.bbox)

def on_release(event):
    print('releasing')
    if on_pick.current is None:
        return
    
    # Reset picked artist state
    art : Artist = on_pick.current
    art.set_animated(False)
    # Get the styles that on_pick.styles changed 
    # and restore each of them using the old
    # values in on_release.style
    try:
        art.set(**on_release.style)
    except RuntimeError as e:
        with out:
            print(e)
    
    ax.draw_artist(art)
    update_bg()
    # fig.canvas.draw_idle()  # Trigger a full redraw to finalize changes
    
    # RESETTING SHOULD BE LAST
    # This is because art = on_pick.current
    # is done for convenience but now its a 
    # reference to on_pick.current.
    # Setting current to None also sets art to None
    on_pick.current = None
    on_release.style = {}

    with out:
        print(f'released artist {type(art)} id:{id(art)}')

def on_draw(event):
    print('drawing')
    # When there is a redraw clear the previuos bg
    global bg
    bg = None
    
with plt.ioff():
    fig, ax = plt.subplots()  
    
x = np.linspace(0,5,30)
y = np.sin(x)
rect = Rectangle([0.5, 0.5], 5, 10, alpha=.3,
                 color = 'yellow', lw = 0)
rect1 = Rectangle([1, 1], 5, 10, alpha=.3,
                 color = 'yellow', lw = 0)
line = Line2D([0,1], [0,1])
rect.set_picker(5)
rect1.set_picker(5)
line.set_picker(5)


# adds the line to the axs objects
# when calling ax.draw() the line 
# is drawn
# NOTE: if line.animated is true then 
# ax.draw() will not draw the line,
# the updates are handled manually
ax.add_artist(rect)
ax.add_artist(rect1)
ax.add_line(line)
ax.relim()
ax.autoscale_view()
fig.tight_layout()
fig.set_size_inches(5,3.5)
fig.canvas.draw()

bg = fig.canvas.copy_from_bbox(ax.bbox)
fig.canvas.blit(ax.bbox)
print(line.get_transform().get_matrix())

# initialize the static variables
on_move.last = (None, None)
on_move.behaviours = {
    Rectangle : update_Rectangle,
    Line2D : update_Line2D
}
on_pick.current = None
on_pick.styles = {
    Rectangle : {
        'alpha' : 0.6,
        'linewidth' : 1
    },
    Line2D : {
        'linewidth' : 2
        },
}


# ___ WIDGETS ___
def on_value_change(change):
    global SKIP_FRAMES
    SKIP_FRAMES = change['new']

def on_check(change):
	print(line.get_transform().get_matrix())
	print(ax.transData.get_matrix())
        
int_range = widgets.IntSlider(value=0, max=10, min=0, description = 'FrameSkip:')
check = widgets.ToggleButtons(options=['normal', 'inverted'],
                              description = 'Frame skipping mode:',
                              tooltips = ['frames skipped', 'frames drawn after every skip'])
        
int_range.observe(on_value_change, names='value')
check.observe(on_check, names='value')
out = widgets.Output()
out.append_stdout('stdout:\n')
out.append_stdout(fig.get_dpi())
right_box = widgets.VBox([check, int_range, out])
app = widgets.HBox([fig.canvas, right_box])


display(app)
print(line.get_transform().get_matrix())
print(ax.transData.get_matrix())

[[68.96931555  0.         53.77456203]
 [ 0.         25.68822697 42.36347854]
 [ 0.          0.          1.        ]]


[[68.96931555  0.         53.77456203]
 [ 0.         25.68822697 42.36347854]
 [ 0.          0.          1.        ]]
[[68.96931555  0.         53.77456203]
 [ 0.         25.68822697 42.36347854]
 [ 0.          0.          1.        ]]


[[99.31581834  0.         77.4353724 ]
 [ 0.         36.9910483  61.00341151]
 [ 0.          0.          1.        ]]
[[99.31581834  0.         77.4353724 ]
 [ 0.         36.9910483  61.00341151]
 [ 0.          0.          1.        ]]
